In [ ]:
%run -i rename.py

In [ ]:
from AiLUT import AiLUT
models = [
  (AiLUT, {}, 'state_dict', '../model/AiLUT/AiLUT-FiveK-sRGB.pth'),
  (AiLUT, {}, 'state_dict', '../model/AiLUT/AiLUT-FiveK-XYZ.pth'),
  (AiLUT, dict(n_ranks=5, backbone='res18'), 'state_dict', '../model/AiLUT/AiLUT-PPR10KA-sRGB.pth')
]
rsts = [
    # match, replace, with, suffix, insert
    (('',), 'backbone.net.', 'backbone.', None, None),
]
subs = []

In [ ]:
from NAFNet import NAFNet
layers = 4
models = [
  (NAFNet, dict(width=32, enc_blk_nums=[1, 1, 1, 28], dec_blk_nums=[1, 1, 1, 1]), 'params', '../model/NAFNet/NAFNet-GoPro-width32.pth'),
  (NAFNet, dict(width=64, enc_blk_nums=[1, 1, 1, 28], dec_blk_nums=[1, 1, 1, 1]), 'params', '../model/NAFNet/NAFNet-GoPro-width64.pth'),
  (NAFNet, dict(width=64, enc_blk_nums=[1, 1, 1, 28], dec_blk_nums=[1, 1, 1, 1]), 'params', '../model/NAFNet/NAFNet-REDS-width64.pth'),
  (NAFNet, dict(width=32, enc_blk_nums=[2, 2, 4, 8], middle_blk_num=12, dec_blk_nums=[2, 2, 2, 2]), 'params', '../model/NAFNet/NAFNet-SIDD-width32.pth'),
  (NAFNet, dict(width=64, enc_blk_nums=[2, 2, 4, 8], middle_blk_num=12, dec_blk_nums=[2, 2, 2, 2]), 'params', '../model/NAFNet/NAFNet-SIDD-width64.pth')
]
rsts = [
    # match, replace, with, suffix, insert
    (('',), 'middle_blks', 'layers.{}'.format(layers), None, None),
]
subs = [
    # match, replace, with, suffix, insert
    (re.compile(r'encoders\.([\d]+\.)(.*)'), r'layers.\1encoder.\2'),
    (re.compile(r'downs\.([\d]+\.)(.*)'), r'layers.\1down.\2'),
    (re.compile(r'ups\.([\d]+)\.(.*)'), lambda m: ''.join(('layers.', str(layers - 1 - int(m[1])), '.up.', m[2]))),
    (re.compile(r'decoders\.([\d]+)\.(.*)'), lambda m: ''.join(('layers.', str(layers - 1 - int(m[1])), '.decoder.', m[2])))
]

In [ ]:
from videoSR import BasicVSRPlusPlus as modules
models = [
  (None, {}, 'state_dict', '../model/vsr/basicvsr_plusplus_c128n25_ntire_decompress_track2_20210314-eeae05e6.pth')
  (None, {}, 'state_dict', '../model/vsr/basicvsr_plusplus_c128n25_ntire_vsr_20210311-1ff35292.pth'),
  (None, {}, 'state_dict', '../model/vsr/basicvsr_plusplus_c64n7_8x1_300k_vimeo90k_bd_20210305-ab315ab1.pth')
]
rsts = [
    # match, replace, with, suffix, insert
    (('',), 'step_counter', '', None, None),
    (('',), 'generator.', '', None, None),
]
subs = [
    # match, replace, with, suffix, insert
    (re.compile(r'(spynet\.basic_module\.[\d]+\.)basic_module\.(.*)'), r'\1\2'),
    (re.compile(r'upconv1\.(.*)'), r'upsample.0.\1'),
    (re.compile(r'upconv2\.(.*)'), r'upsample.3.\1'),
    (re.compile(r'conv_hr\.(.*)'), r'upsample.6.\1'),
    (re.compile(r'conv_last\.(.*)'), r'upsample.8.\1')
]

In [ ]:
from DLN import DLN
models = [
    (DLN, {}, 0, '../model/DLN/DLN_finetune_LOL.pth')
]
rsts = [
    (('',), 'module.', '', None, None)
]

In [ ]:
from models import Net2x, Net3x, Net4x, NetDN
models = [
    # model Class, kwargs, root, *paths
    (Net2x, {}, 0, '../model/a2/model_new.pth', '../model/p2/model_new.pth'),
    (Net3x, {}, 0, '../model/a3/model_new.pth', '../model/p3/model_new.pth'),
    (Net4x, {}, 0, '../model/a4/model_new.pth', '../model/p4/model_new.pth'),
    (NetDN, {}, 0, '../model/dn_lite5/model_new.pth', '../model/dn_lite10/model_new.pth', '../model/dn_lite15/model_new.pth')
]
rsts = [
    # match, replace, with, suffix, insert
    ((r'convt_F.\.',), None, None, '', '0.')
]

In [ ]:
from models import RRDBNet
models = [
    (RRDBNet, {'num_in_ch': 3, 'num_out_ch': 3, 'scale': 2}, 'params_ema', '../model/gan/RealESRGAN_x2plus.pth'),
    (RRDBNet, {'num_in_ch': 3, 'num_out_ch': 3}, 'params_ema', '../model/gan/RealESRGAN_x4plus.pth'),
    (RRDBNet, {'num_in_ch': 3, 'num_out_ch': 3, 'num_block': 6}, 'params_ema', '../model/gan/RealESRGAN_x4plus_anime_6B.pth')
]
subs = [
    (re.compile(r'(body.[\d]+.rdb[\d]+.)conv([\d]+)(\..*)'), lambda m: ''.join((m[1], 'conv.', str(int(m[2]) - 1), m[3])))
]

In [ ]:
from videoSR import modules
models = [
  (None, {}, 'params', '../model/vsr/IconVSR_Vimeo90K_BDx4-cfcb7e00.pth')
]
rsts = [
    # match, replace, with, suffix, insert
    ((r'[^\.]+_trunk\.',), 'main.', '', None, None),
]
subs = [
    # match, replace, with, suffix, insert
    (re.compile(r'(spynet\.basic_module\.[\d]+\.)basic_module\.(.*)'), r'\1\2'),
    (re.compile(r'upconv1\.(.*)'), r'upsample.0.\1'),
    (re.compile(r'upconv2\.(.*)'), r'upsample.3.\1'),
    (re.compile(r'conv_hr\.(.*)'), r'upsample.6.\1'),
    (re.compile(r'conv_last\.(.*)'), r'upsample.8.\1')
]

In [ ]:
from ESTRNN import modules

models = [
  # model Class, kwargs, root, *paths
  (None, {}, 'state_dict', '../model/ESTRNN/ESTRNN_C80B15_BSD_1ms8ms.tar', '../model/ESTRNN/ESTRNN_C80B15_BSD_2ms16ms.tar', '../model/ESTRNN/ESTRNN_C80B15_BSD_3ms24ms.tar')
]
rsts = [
  # match, replace, with, suffix, insert
  (('',), 'module.model.', '', None, None),
  (('recons.',), 'model.', '', None, None)
]
subs = [
  (re.compile(r'(.+\..+\..+)\.conv1x1(\..*)'), r'\1.3\2'),
  (re.compile(r'(.+\.)dense_layers\.(.*)'), r'\1\2'),
  (re.compile(r'(cell\.F_B[\d]+)\.rdb(\..*)'), r'\1.0\2'),
  (re.compile(r'(cell\.F_B[\d]+)\.down_sampling(\..*)'), r'\1.1\2')
]

In [ ]:
from IFRNet import modules, getOpt

models = [
  # model Class, kwargs, root, *paths
  (None, {}, None, '../model/IFRNet/IFRNet_S_Vimeo90K.pth', '../model/IFRNet/IFRNet_Vimeo90K.pth', '../model/IFRNet/IFRNet_L_Vimeo90K.pth')
]
rsts = [
  ((r'decoder[\d]+\.',), 'convblock.', '', None, None)
]
subs = [
  (re.compile(r'encoder\.pyramid([\d]+)\.'), lambda m: 'encoder.pyramids.{}.'.format(int(m[1]) - 1)),
  (re.compile(r'decoder([\d]+)\.'), lambda m: 'decoder.decoders.{}.'.format(4 - int(m[1])))
]
options = [
  {'model': 'S', 'sf': 2},
  {'model': 'M', 'sf': 2},
  {'model': 'L', 'sf': 2}
]

In [ ]:
from MPRNet import MPRNet
models = [
    # model Class, kwargs, root, *paths
    (MPRNet, dict(n_feat=96, scale_unetfeats=48, scale_orsnetfeats=32), 'state_dict', '../model/MPRNet/model_deblurring.pth'),
    (MPRNet, dict(n_feat=80, scale_unetfeats=48, scale_orsnetfeats=32), 'state_dict', '../model/MPRNet/model_denoising.pth'),
    (MPRNet, dict(n_feat=40, scale_unetfeats=20, scale_orsnetfeats=16), 'state_dict', '../model/MPRNet/model_deraining.pth')
]
rsts = [
    ((r'(.+\.)body\.',), 'body.', '', None, None),
    ((r'(.+\.)CA\.',), 'CA.', '3.', None, None),
]
sm1 = lambda s: str(int(s) - 1)
subs = [
    (re.compile(r'shallow_feat([\d]+)(\..*)'), lambda m: ''.join(('shallow_feat.', sm1(m[1]), m[2]))),
    (re.compile(r'(sam|concat)([\d])[\d](\..*)'), lambda m: ''.join((m[1], '.', sm1(m[2]), m[3]))),
    (re.compile(r'stage3_orsnet\.orb([\d]+)(\..*)'), lambda m: ''.join(('stage3_orsnet.orb.', sm1(m[1]), m[2]))),
    (re.compile(r'stage3_orsnet\.up_(enc|dec)1\.up(\..*)'), lambda m: ''.join(('stage3_orsnet.conv_', m[1], '.1.0', m[2]))),
    (re.compile(r'stage3_orsnet\.up_(enc|dec)([\d]+)\.([\d]+)\.up(\..*)'), lambda m: ''.join(('stage3_orsnet.conv_', m[1], '.', m[2], '.', m[3], m[4]))),
    (re.compile(r'stage3_orsnet\.conv_(enc|dec)([\d]+)(\..*)'), lambda m: ''.join(('stage3_orsnet.conv_', m[1], '.', sm1(m[2]), '.', sm1(m[2]), m[3]))),
    (re.compile(r'stage3_orsnet\.(.*)'), r'encoder.2.\1'),
    (re.compile(r'(stage[\d]+_encoder\.encoder_level[\d]+\.)([\d]+)(\..*)'), lambda m: ''.join((m[1], str(int(m[2]) + 1), m[3]))),
    (re.compile(r'stage([\d]+)_(encoder|decoder)\.(encoder|decoder)_level([\d]+)(\..*)'), lambda m: ''.join((m[2], '.', sm1(m[1]), '.', m[3], '.', sm1(m[4]), m[5]))),
    (re.compile(r'stage([\d]+)_encoder\.down([\d])[\d]\.down(\..*)'), lambda m: ''.join(('encoder.', sm1(m[1]), '.encoder.', m[2], '.0', m[3]))),
    (re.compile(r'stage([\d]+)_encoder\.(csff_enc|csff_dec)([\d]+)(\..*)'), lambda m: ''.join(('encoder.', sm1(m[1]), '.', m[2], '.', sm1(m[3]), m[4]))),
    (re.compile(r'([^\.]+\.)skip_attn([\d]+)(\..*)'), lambda m: ''.join((m[1], 'skip_attn.', sm1(m[2]), m[3]))),
    (re.compile(r'(stage[\d]+_decoder\.up)[\d]([\d])(\..*)'), lambda m: ''.join((m[1], '.', sm1(m[2]), m[3]))),
    (re.compile(r'stage([\d])_(encoder|decoder)(\..*)'), lambda m: ''.join((m[2], '.', sm1(m[1]), m[3])))
]

In [ ]:
m = renameByRules(models, rsts=rsts, subs=subs, rules=[o['weight'] for o in modules.values()] if modules else None)

In [ ]:
m = torch.load('./RealBasicVSR_x4.pth')['state_dict']
state = {}
for key in getRoot(m, 'generator.image_cleaning.'):
  newKey = key[len(r):]
  state[newKey] = m[key]
cc(state, getSubNames)([('0.main.', '0.')])

In [ ]:
pp(m)

In [ ]:
import os
import os.path as osp
os.chdir(osp.join(osp.dirname(osp.abspath(__file__)), '../'))
for option in options:
  print(option)
  getOpt(option)

In [ ]:
pp(m)

for key, o in modules.items():
  m1 = o['f']()
  print(key, m1.load_state_dict(m[key]))

In [ ]:
from models import conv2d311, ResidualBlockNoBN, make_layer
from torch.nn import Sequential, LeakyReLU

ResidualBlocksWithInputConv = lambda in_channels, out_channels=64, num_blocks=30: Sequential(
  conv2d311(in_channels, out_channels),
  LeakyReLU(negative_slope=0.1, inplace=True),
  make_layer(ResidualBlockNoBN, num_blocks, num_feat=out_channels)
)
ImageCleaning = lambda num_feat=64, num_cleaning_blocks=20: Sequential(
  ResidualBlocksWithInputConv(3, num_feat, num_cleaning_blocks),
  conv2d311(num_feat, 3)
)
m = ImageCleaning()
m.load_state_dict(state)
torch.save(m.state_dict(), '../model/vsr/RealBasicVSR_ImageCleaning.pth')

In [ ]:
modelPath = models[-1][-1] + '.new'

In [ ]:
modelPath = models[-1][-1]
m = models[-1][0](**models[-1][1])
m.load_state_dict(torch.load(modelPath))

In [ ]:
for param in m.parameters():
  param.requires_grad_(False)
m = castModel(m.eval())
opt = Option(modelPath)
opt.scale = 1
opt.padding = 5
opt.align = 16
opt.ramCoef = 1 / 1.
opt.squeeze = lambda x: x.squeeze(0)
opt.unsqueeze = lambda x: x.unsqueeze(0)
opt.modelCached = m
modelPath

In [ ]:
original = readPic('./blur.png')
show(original)

In [ ]:
y = ensemble(opt)(original)
show(y)

In [ ]:
from torch.profiler import profile, ProfilerActivity, schedule
N = 41
with profile(
  activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
  schedule=schedule(wait=2, warmup=9, active=30, repeat=1)) as p:
  for iter in range(N):
    # y = m1(x)
    p.step()
  print(p.key_averages().table(sort_by="self_cuda_time_total", row_limit=-1))